# LightFM model for influence marketing

Description

### Model theoretical explanation
This model is based in ...

### 1. Import Libraries

In [2]:
# Install all the libraries in requirements.txt
import sys
import os

import itertools
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scrapbook as sb
import requests
import io

import lightfm
from lightfm import LightFM
from lightfm.data import Dataset
from lightfm import cross_validation

# Import LightFM's evaluation metrics
from lightfm.evaluation import precision_at_k as lightfm_prec_at_k
from lightfm.evaluation import recall_at_k as lightfm_recall_at_k

# Import repo's evaluation metrics
from recommenders.evaluation.python_evaluation import precision_at_k, recall_at_k

from recommenders.utils.timer import Timer
from recommenders.datasets import movielens
from recommenders.models.lightfm.lightfm_utils import (
    track_model_metrics, prepare_test_df, prepare_all_predictions,
    compare_metric, similar_users, similar_items)

print("System version: {}".format(sys.version))
print("LightFM version: {}".format(lightfm.__version__))

c:\Users\manue\AppData\Local\Programs\Python\Python38\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


System version: 3.8.7 (tags/v3.8.7:6503f05, Dec 21 2020, 17:59:51) [MSC v.1928 64 bit (AMD64)]
LightFM version: 1.16


### 2. Defining Variables

In [3]:
# Select MovieLens data size
MOVIELENS_DATA_SIZE = '100k'

# default number of recommendations
K = 10
# percentage of data used for testing
TEST_PERCENTAGE = 0.25
# model learning rate
LEARNING_RATE = 0.25
# no of latent factors
NO_COMPONENTS = 20
# no of epochs to fit model
NO_EPOCHS = 20
# no of threads to fit model
NO_THREADS = 32
# regularisation for both user and item features
ITEM_ALPHA = 1e-6
USER_ALPHA = 1e-6

# seed for pseudonumber generations
SEED = 42

### 3. Retrive Data

In [4]:
data = pd.read_csv('df.csv')
data

,Account,Link,Followers,Audience Country,Authentic engagement,Engagement avg,Category1,Hashtags,Cost Story,Cost Post,Name_co,Category_co,Hashtags_co,Country_co,Followers_co,num_coincidentes,Puntos
0,cristiano,https://www.instagram.com/cristiano/,400100000.0,India,7800000.0,9500000.0,Sports with a ball,"['#sunset', '#wanderlust', '#weekend', '#sky',...",468000.0,1092000.0,Firefly Bakery,Sports with a ball,"['#modeling', '#flex', '#beautifuldestinations...",India,222874,0,15
1,cristiano,https://www.instagram.com/cristiano/,400100000.0,India,7800000.0,9500000.0,Sports with a ball,"['#sunset', '#wanderlust', '#weekend', '#sky',...",468000.0,1092000.0,The Espresso Bar,Family,"['#football', '#endurance']",France,580698,0,0
2,cristiano,https://www.instagram.com/cristiano/,400100000.0,India,7800000.0,9500000.0,Sports with a ball,"['#sunset', '#wanderlust', '#weekend', '#sky',...",468000.0,1092000.0,Coastal Cafe,Extreme,"['#getfit', '#fitlife', '#familytime', '#telec...",Turkey,439532,0,0
3,cristiano,https://www.instagram.com/cristiano/,400100000.0,India,7800000.0,9500000.0,Sports with a ball,"['#sunset', '#wanderlust', '#weekend', '#sky',...",468000.0,1092000.0,The Garden Shed,Management,"['#coffee', '#foodaddict', '#selfie', '#shoppi...",Ethiopia,61082,0,0
4,cristiano,https://www.instagram.com/cristiano/,400100000.0,India,7800000.0,9500000.0,Sports with a ball,"['#sunset', '#wanderlust', '#weekend', '#sky',...",468000.0,1092000.0,The Seafood Shack,Food,"['#photo', '#naturephotography', '#holidayseas...",Kenya,407614,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,eduincaz,https://www.instagram.com/eduincaz/,6200000.0,Mexico,305600.0,391900.0,Lifestyle,"['#foodstagram', '#smilemore', '#foodgrammer',...",18336.0,42784.0,The Winding Road,Fashion,"['#foodbloggerlife', '#happyholidays', '#smile...",Indonesia,211541,0,0
99996,eduincaz,https://www.instagram.com/eduincaz/,6200000.0,Mexico,305600.0,391900.0,Lifestyle,"['#foodstagram', '#smilemore', '#foodgrammer',...",18336.0,42784.0,Blue Ribbon Bakery,Fashion,"['#loveit', '#foodexplorer']",United Kingdom,495099,0,0
99997,eduincaz,https://www.instagram.com/eduincaz/,6200000.0,Mexico,305600.0,391900.0,Lifestyle,"['#foodstagram', '#smilemore', '#foodgrammer',...",18336.0,42784.0,The Rustic Lantern,Modeling,"['#marathon', '#foodieforlife', '#blessings', ...",Colombia,102789,0,0
99998,eduincaz,https://www.instagram.com/eduincaz/,6200000.0,Mexico,305600.0,391900.0,Lifestyle,"['#foodstagram', '#smilemore', '#foodgrammer',...",18336.0,42784.0,Vintage Vineyards,Undefined,"['#foodheaven', '#fashionable', '#outfit', '#f...",United Kingdom,26754,0,0


### 4. Prepare Data

In [5]:
dataset = Dataset()

In [15]:
dataset.fit(data['Name_co'], 
            data['Account'], 
            item_features=('Audience Country','Authentic engagement', 'Category1', 'Hashtags'),
            user_features=('Category_co', 'Hashtags_co', 'Country_co', 'Followers_co'))

In [16]:
# quick check to determine the number of unique users and items in the data
num_users, num_topics = dataset.interactions_shape()
print(f'Num users: {num_users}, num_topics: {num_topics}.')

Num users: 99, num_topics: 1000.


In [17]:
item_features = dataset.build_item_features((x, y) for x,y in zip(data.Account, data['Audience Country'], data['Authentic engagement'], data['Category1'], data['Hashtags']))

ValueError: too many values to unpack (expected 2)